### Load the necessary modules

In [ ]:
from pymatgen.ext.matproj import MPRester
from atomate.vasp.powerups import add_additional_fields_to_taskdocs,set_execution_options
from atomate.vasp.workflows.base.lobster import get_wf_lobster_test_basis
from fireworks import LaunchPad
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('material_ids.csv') # Load the csv file consisting of mp-ids

In [ ]:
mpid = df['metadata.material_id'].values ## store it in a python list

In [ ]:
m= MPRester() # initialize materails project api module

In [ ]:
for i in mpid:
    struct=m.get_structure_by_material_id(i) # fetche the structure using material id 
    wf = get_wf_lobster_test_basis(structure=struct, 
                                   user_incar_settings = {"NEDOS":10000, "ICHARG":2}, #modify vasp static run settings 
                                   user_kpoints_settings= {"grid_density":6000}, #sets k-point grid
                                   user_lobsterin_settings={"LSODOS":True}, # makes lobster calculate LSO dos
                                   calculation_type='standard_with_energy_range_from_vasprun') 
    update_dict={"material-id": i} 

    #This is a powerup to add the material-id to the database
    wf=add_additional_fields_to_taskdocs(wf,update_dict=update_dict, task_name_constraint='VaspToDb')
    wf=add_additional_fields_to_taskdocs(wf,update_dict=update_dict, task_name_constraint='LobsterRunToDb')

    wf=set_execution_options(wf, fworker_name="supermuc_lobster", category=None, fw_name_constraint="lobster_calculation", task_name_constraint=None)
    wf=set_execution_options(wf, fworker_name="supermuc_delete", category=None, fw_name_constraint="DelteWavecar", task_name_constraint=None)


    lpad = LaunchPad.auto_load() # loads this based on the FireWorks configuration
    lpad.add_wf(wf) #adds workflow to firework database